In [1]:
import numpy as np
import pandas as pd
import feather
from pandasql import sqldf
from fisher import pvalue
from scipy import stats
from pprint import pprint

In [21]:
def toy_multi_gene_cnv(p, n_cnv, n_max=10, causal=5, const=1, seed=1):
    np.random.seed(seed)
    cnv_len = np.random.choice((np.random.geometric(p, size=n_cnv) - 1), n_cnv, replace=False)
    cnv_len = cnv_len[cnv_len <= 10].tolist()
    start_pos = [np.random.choice(range(n_max+1-i)) if i!=0 else -1 for i in cnv_len]
    ptn_ls = []
    for j,i in enumerate(start_pos):
        if i == -1:
            ptn = [0]*n_max
        elif i == 0:
            ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-i)
        else:
            ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-(i-1))
        ptn_ls.append(ptn)
    mat = [[np.random.normal(0,1)] + line for line in ptn_ls]
    config = []
    for line in mat:
#         config.append(line[0]+const) if line[causal-1] == 1 else config.append(line[0])
        if line[causal] == 1:
            line[0] = line[0] + const
        config.append(line[0])

#         line[0] = 1 if line[0] > np.median(config) else 0
        if line[0] > np.median(config):
            line[0] = 1
        else:
            line[0] = 0
    mat = np.matrix(mat)
    df = pd.DataFrame(mat, columns = ["phenotype"] + ["gene{}".format(i+1) for i in range(mat.shape[1]-1)])
    counts = df.groupby(["gene{}".format(i+1) for i in range(mat.shape[1]-1)] + ["phenotype"]).size()
    for col in df.columns:
        df[col] = df[col].astype(np.uint8)
    output = feather.write_dataframe(df, "data/toy_n{}_p{}_causal{}.feather".format(len(cnv_len), p, causal))
    return df, counts

In [22]:
df, counts = toy_multi_gene_cnv(p=0.5, n_cnv=2000)
# print (df["phenotype"].tolist())

In [4]:
p = 0.3
n_cnv = 200
n_max = 10
const = 1
seed = 9
np.random.seed(seed)
cnv_len = np.random.choice((np.random.geometric(p, size=n_cnv) - 1), n_cnv, replace=False)
cnv_len = cnv_len[cnv_len <= 10].tolist()
print (len(cnv_len))

199


In [5]:
start_pos = [np.random.choice(range(n_max+1-i)) if i!=0 else -1 for i in cnv_len]

In [6]:
ptn_ls = []
for j,i in enumerate(start_pos):
    if i == -1:
        ptn = [0]*n_max
    elif i == 0:
        ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-i)
    else:
        ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-(i-1))
    ptn_ls.append(ptn)

In [7]:
np.random.seed(seed)
mat = [[np.random.normal(0,1)] + line for line in ptn_ls]
mat_test = mat
print (mat_test==mat)

True


In [8]:
config = []
causal = 5
const = 1
for line in mat:
###     config.append(line[0]+const) if line[causal-1] == 1 else config.append(line[0])
    if line[causal-1] == 1:
        line[0] = line[0] + const
        config.append(line[0])
    else:
        config.append(line[0])
    if line[0] > np.median(config):
        line[0] = 1
    else:
        line[0] = 0
#     line[0] = 1 if line[0] > np.median(config) else 0
mat1 = np.matrix(mat)

In [9]:
pprint ((mat1))

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])


In [10]:
for line in mat_test:
    if line[causal-1] == 1:
        line[0] = line[0] + const
        config.append(line[0])
    else:
        config.append(line[0])
    if line[0] > np.median(config):
        line[0] = 1
    else:
        line[0] = 0
mat2 = np.matrix(mat_test)